# Marker PDF to Markdown - Building Code

**사용법:**
1. Runtime > Change runtime type > **T4 GPU** 선택
2. 셀 순서대로 실행 (Shift+Enter)
3. PDF 업로드하면 자동 변환

**느리면?**
- Runtime > Disconnect and delete runtime > 다시 연결

**큰 PDF (200+ 페이지):**
- 셀 5A: 일반 변환
- 셀 5B: 페이지 분할 변환 (메모리 부족시)

In [ ]:
# 셀 1: GPU 확인
import torch

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    print(f'GPU: {gpu_name}')
    print('GPU 준비 완료!')
else:
    print('GPU 없음!')
    print('Runtime > Change runtime type > T4 GPU 선택하세요')

In [ ]:
# 셀 2: 빠른 다운로드 설정 + 설치
import os

# 빠른 다운로드 활성화
os.environ['HF_HUB_ENABLE_HF_TRANSFER'] = '1'

# 설치
!pip install marker-pdf pymupdf hf_transfer -q

print('설치 완료!')

In [ ]:
# 셀 3: 모델 미리 다운로드 (진행상황 표시)
# 처음 1회만 오래 걸림, 이후는 캐시됨

print('모델 다운로드 중... (처음만 오래 걸림)')
print('속도가 너무 느리면: Runtime > Disconnect and delete runtime > 재연결')
print('='*50)

# Marker import하면 자동으로 모델 다운로드
from marker.converters.pdf import PdfConverter
from marker.models import create_model_dict

print('모델 로드 중...')
models = create_model_dict()

print('='*50)
print('모델 준비 완료!')

In [ ]:
# 셀 4: PDF 업로드 + 페이지 수 확인
from google.colab import files
import fitz  # PyMuPDF

print('파일 선택 창이 열립니다...')
uploaded = files.upload()

if uploaded:
    filename = list(uploaded.keys())[0]
    doc = fitz.open(filename)
    page_count = len(doc)
    doc.close()
    
    print(f'\n업로드 완료: {filename}')
    print(f'페이지 수: {page_count}')
    print('='*50)
    
    if page_count > 200:
        print('페이지가 많습니다! 셀 5B (분할 변환) 권장')
    else:
        print('셀 5A (일반 변환) 실행하세요')
else:
    print('파일이 업로드되지 않았습니다')

In [ ]:
# 셀 5A: 일반 변환 (200페이지 이하 권장)
import time
start = time.time()

!marker_single "{filename}" --output_dir ./output

elapsed = time.time() - start
print('='*50)
print(f'변환 완료! ({elapsed/60:.1f}분 소요)')
!ls -la output/

In [ ]:
# 셀 5B: 페이지 분할 변환 (큰 PDF용)
# 메모리 크래시 방지를 위해 50페이지씩 나눠서 처리

import fitz
import os
import shutil
import time
import gc
import torch

CHUNK_SIZE = 50  # 한 번에 처리할 페이지 수 (메모리 부족시 30으로 줄이기)

start_total = time.time()

# 폴더 준비
os.makedirs('chunks', exist_ok=True)
os.makedirs('output', exist_ok=True)

# PDF 분할
doc = fitz.open(filename)
total_pages = len(doc)
chunks = []

print(f'총 {total_pages} 페이지를 {CHUNK_SIZE}페이지씩 분할...')

for start in range(0, total_pages, CHUNK_SIZE):
    end = min(start + CHUNK_SIZE, total_pages)
    chunk_name = f'chunks/chunk_{start+1:04d}_{end:04d}.pdf'
    
    chunk_doc = fitz.open()
    chunk_doc.insert_pdf(doc, from_page=start, to_page=end-1)
    chunk_doc.save(chunk_name)
    chunk_doc.close()
    
    chunks.append(chunk_name)
    print(f'  생성: {chunk_name} ({end-start}페이지)')

doc.close()
print(f'\n총 {len(chunks)}개 청크 생성 완료')
print('='*50)

# 각 청크 변환
all_markdown = []

for i, chunk in enumerate(chunks):
    start_chunk = time.time()
    print(f'\n[{i+1}/{len(chunks)}] {chunk} 변환 중...')
    chunk_output = f'output/chunk_{i:03d}'
    
    !marker_single "{chunk}" --output_dir "{chunk_output}"
    
    # 변환된 마크다운 읽기
    import glob
    md_files = glob.glob(f'{chunk_output}/**/*.md', recursive=True)
    if md_files:
        with open(md_files[0], 'r', encoding='utf-8') as f:
            content = f.read()
        all_markdown.append(f'\n\n<!-- Chunk: {chunk} -->\n\n{content}')
        elapsed_chunk = time.time() - start_chunk
        print(f'  완료! ({elapsed_chunk/60:.1f}분)')
    
    # 메모리 정리
    gc.collect()
    torch.cuda.empty_cache()

# 마크다운 합치기
combined_name = filename.replace('.pdf', '_combined.md').replace('.PDF', '_combined.md')
with open(f'output/{combined_name}', 'w', encoding='utf-8') as f:
    f.write('\n'.join(all_markdown))

elapsed_total = time.time() - start_total
print('='*50)
print(f'모든 변환 완료! (총 {elapsed_total/60:.1f}분 소요)')
print(f'합쳐진 파일: output/{combined_name}')
!ls -la output/

In [ ]:
# 셀 6: 결과 미리보기 (선택사항)
import glob
import os

md_files = glob.glob('output/**/*.md', recursive=True)
if md_files:
    # 가장 큰 파일 선택
    md_file = max(md_files, key=lambda x: os.path.getsize(x))
    with open(md_file, 'r', encoding='utf-8') as f:
        content = f.read()
    print(f'파일: {md_file}')
    print(f'크기: {len(content):,} 글자')
    print('='*50)
    print(content[:3000])
    print('\n... (생략) ...')
else:
    print('변환된 파일이 없습니다. 셀 5A 또는 5B를 먼저 실행하세요.')

In [ ]:
# 셀 7: 결과 다운로드
!zip -r result.zip output/

from google.colab import files
files.download('result.zip')

print('다운로드 시작됨!')

In [ ]:
# 셀 8: 임시 파일 정리 (선택사항)
!rm -rf chunks/
print('청크 파일 삭제 완료')